In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import base64
#!pip install spotipy
import requests
from bs4 import BeautifulSoup
import time
from spotipy.oauth2 import SpotifyClientCredentials
#!pip install spotipy --upgrade
import re

In [2]:
charts_df = pd.read_csv('../data/charts.csv')
lyrics_df = pd.read_csv('../data/billboard_24years_lyrics_spotify.csv')
spotify_qualities_df = pd.read_csv('../data/Hot100.csv')
song_attributes_df = pd.read_csv('../data/songAttributes_1999-2019.csv')

In [14]:
#spotify_qualities_df.info()
updated_song_attributes_df = song_attributes_df.drop(['Unnamed: 0', 'Popularity', 'Album'], axis=1)
updated_song_attributes_df = updated_song_attributes_df.rename(columns={'Name': 'song', 'Artist': 'artist'})
updated_song_attributes_df['artist'] = updated_song_attributes_df['artist'].str.lower()
updated_song_attributes_df['song'] = updated_song_attributes_df['song'].str.lower()
updated_song_attributes_df.head(20)

,Acousticness,artist,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,song,Speechiness,Tempo,TimeSignature,Valence
0,0.000728,collective soul,0.520,234947,0.904,False,0.010300,0.0634,-5.030,1,welcome all again,0.0309,106.022,4,0.365
1,0.018200,collective soul,0.581,239573,0.709,False,0.000664,0.1740,-4.909,1,fuzzy,0.0282,120.027,4,0.408
2,0.000473,collective soul,0.572,198400,0.918,False,0.000431,0.0977,-3.324,0,dig,0.0559,144.061,4,0.370
3,0.000970,collective soul,0.596,231453,0.661,False,0.000033,0.1130,-5.051,1,you,0.0254,111.975,4,0.183
4,0.000036,collective soul,0.520,222520,0.808,False,0.000010,0.0800,-4.553,0,my days,0.0318,92.721,4,0.666
5,0.010600,collective soul,0.353,263880,0.754,False,0.000073,0.1200,-4.570,1,understanding,0.0452,138.003,4,0.546
6,0.043600,collective soul,0.629,214973,0.787,False,0.000047,0.1000,-5.657,1,staring down,0.0277,100.440,4,0.476
7,0.000402,collective soul,0.646,206067,0.853,False,0.000090,0.1170,-4.685,1,she does,0.0278,127.963,4,0.699
8,0.000521,collective soul,0.645,216400,0.840,False,0.000011,0.0370,-4.995,1,lighten up,0.0697,140.025,4,0.795
9,0.000380,collective soul,0.601,212013,0.861,False,0.000002,0.1310,-4.690,1,love,0.0319,120.017,4,0.615


In [37]:
updated_song_attributes_df = updated_song_attributes_df.drop_duplicates(subset=['artist', 'song'])
updated_song_attributes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130191 entries, 0 to 154930
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Acousticness      130191 non-null  float64
 1   artist            130191 non-null  object 
 2   Danceability      130191 non-null  float64
 3   Duration          130191 non-null  int64  
 4   Energy            130191 non-null  float64
 5   Explicit          130191 non-null  bool   
 6   Instrumentalness  130191 non-null  float64
 7   Liveness          130191 non-null  float64
 8   Loudness          130191 non-null  float64
 9   Mode              130191 non-null  int64  
 10  song              130191 non-null  object 
 11  Speechiness       130191 non-null  float64
 12  Tempo             130191 non-null  float64
 13  TimeSignature     130191 non-null  int64  
 14  Valence           130191 non-null  float64
 15  song_cleaned      130191 non-null  object 
dtypes: bool(1), float64(9), i

In [16]:
updated_spotify_qualities_df = spotify_qualities_df.drop(['Popularity', 'Album'], axis=1)
updated_spotify_qualities_df = updated_spotify_qualities_df.rename(columns={'Artist': 'main_artist', 'Track': 'song', 'Year': 'year'})
updated_spotify_qualities_df['main_artist'] = updated_spotify_qualities_df['main_artist'].str.lower()
updated_spotify_qualities_df['song'] = updated_spotify_qualities_df['song'].str.lower()
updated_spotify_qualities_df.tail(20)

,song,main_artist,year,Duration,Time_Signature,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
600,perreo porky,yng lvcas,2023,224842,4,0.887,0.5980,6,-6.890,1,0.1000,0.0990,0.001790,0.1010,0.768,95.010
601,ven hdp,yng lvcas,2023,154666,4,0.831,0.8290,6,-5.887,1,0.0873,0.4500,0.001400,0.0915,0.865,96.969
602,chicken fried,zac brown band,2008,238146,4,0.566,0.7130,6,-4.250,1,0.0418,0.6450,0.000000,0.1140,0.805,169.861
603,colder weather,zac brown band,2010,273893,4,0.667,0.3810,3,-7.494,1,0.0294,0.4550,0.000000,0.1160,0.199,135.975
604,goodbye in her eyes,zac brown band,2012,324266,4,0.411,0.5580,3,-7.393,1,0.0380,0.1620,0.000004,0.0879,0.379,81.274
605,highway 20 ride,zac brown band,2008,229280,4,0.531,0.4030,3,-7.195,1,0.0338,0.5510,0.000014,0.1050,0.431,76.125
606,homegrown,zac brown band,2015,205000,4,0.712,0.8070,6,-3.976,1,0.0297,0.0244,0.000122,0.0905,0.936,105.024
607,jump right in,zac brown band,2013,180813,4,0.699,0.8190,1,-5.111,1,0.0580,0.0886,0.000000,0.1430,0.654,104.020
608,knee deep (feat. jimmy buffett),zac brown band,2010,203173,4,0.715,0.5840,8,-8.231,1,0.0296,0.4320,0.000001,0.0950,0.609,90.990
609,toes,zac brown band,2008,262773,4,0.801,0.7590,11,-5.398,1,0.0378,0.6570,0.000000,0.0897,0.821,129.971


In [9]:
charts_df['date'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['date'].dt.year
updated_charts_df = charts_df.drop(['date', 'last-week', 'rank'], axis=1)
updated_charts_df['artist'] = updated_charts_df['artist'].str.lower()
updated_charts_df['song'] = updated_charts_df['song'].str.lower()
updated_charts_df['year'].max()

2021

In [12]:
featured_artists= ['featuring', '&', 'with', 'x']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

updated_charts_df['featured_artist'] = updated_charts_df['artist'].apply(extract_featured)
updated_charts_df['main_artist'] = updated_charts_df['artist'].apply(extract_main_artist)
updated_charts_df.tail(20)

,song,artist,peak-rank,weeks-on-board,year,featured_artist,main_artist
330067,beautiful delilah,chuck berry,81,1,1958,None,chuck berry
330068,blip blop,bill doggett,82,1,1958,None,bill doggett
330069,try the impossible,lee andrews and the hearts,83,1,1958,None,lee andrews and the hearts
330070,summertime blues,eddie cochran,84,1,1958,None,eddie cochran
330071,got a match?,the daddy-o's,85,1,1958,None,the daddy-o's
330072,to be loved,jackie wilson,86,1,1958,None,jackie wilson
330073,jealousy,kitty wells,87,1,1958,None,kitty wells
330074,just like in the movies,the upbeats,88,1,1958,None,the upbeats
330075,blue boy,jim reeves,89,1,1958,None,jim reeves
330076,stay,the ames brothers,90,1,1958,None,the ames brothers


In [11]:
range_updated_charts_df = updated_charts_df[
    (updated_charts_df['year'] >= 1999) &
    (updated_charts_df['year'] <= 2021)]
range_updated_charts_df.tail(20)

,song,artist,peak-rank,weeks-on-board,year,featured_artist,main_artist
119280,angel in disguise,brandy,75,5,1999,None,brandy
119281,i don't want to miss a thing,aerosmith,1,18,1999,None,aerosmith
119282,father of mine,everclear,77,5,1999,None,everclear
119283,never there,cake,81,5,1999,None,cake
119284,it must be love,ty herndon,38,5,1999,None,ty herndon
119285,faded pictures,case & joe,86,2,1999,joe,case
119286,home alone,r. kelly featuring keith murray,65,5,1999,keith murray,r. kelly
119287,hooch,everything,69,5,1999,None,everything
119288,we really shouldn't be doing this,george strait,44,5,1999,None,george strait
119289,fire escape,fastball,86,5,1999,None,fastball


In [29]:
filtered_artist_df = range_updated_charts_df[range_updated_charts_df['main_artist'].isin(updated_song_attributes_df['artist'])]
filtered_artist_df.head(50)

,song,artist,peak-rank,weeks-on-board,year,featured_artist,main_artist
0,easy on me,adele,1,3,2021,None,adele
4,bad habits,ed sheeran,2,18,2021,None,ed sheeran
5,way 2 sexy,drake featuring future & young thug,1,8,2021,future & young thug,drake
6,shivers,ed sheeran,7,7,2021,None,ed sheeran
21,knife talk,drake featuring 21 savage & project pat,4,8,2021,21 savage & project pat,drake
25,girls want girls,drake featuring lil baby,2,8,2021,lil baby,drake
36,ghost,justin bieber,32,5,2021,None,justin bieber
43,memory i don't mess with,lee brice,33,14,2021,None,lee brice
46,sharing locations,meek mill featuring lil baby & lil durk,22,9,2021,lil baby & lil durk,meek mill
51,fair trade,drake featuring travis scott,3,8,2021,travis scott,drake


In [34]:
updated_song_attributes_df[updated_song_attributes_df['song'] == 'made for you']

,Acousticness,artist,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,song,Speechiness,Tempo,TimeSignature,Valence,song_cleaned
124844,0.137,onerepublic,0.637,257493,0.557,False,0.020100,0.0977,-10.196,1,made for you,0.0410,122.044,4,0.400,made for you
124918,0.140,onerepublic,0.640,257493,0.555,False,0.019200,0.0981,-10.204,1,made for you,0.0411,122.038,4,0.395,made for you
147995,0.770,jake owen,0.581,238400,0.441,False,0.000028,0.1110,-6.829,1,made for you,0.0268,82.125,4,0.337,made for you


In [40]:
another_try_df = pd.merge(range_updated_charts_df, updated_song_attributes_df, left_on=['main_artist','song'], right_on=['artist', 'song_cleaned'], how='inner')
another_try_df.head(30)

,song_x,artist_x,peak-rank,weeks-on-board,year,featured_artist,main_artist,Acousticness,artist_y,Danceability,...,Instrumentalness,Liveness,Loudness,Mode,song_y,Speechiness,Tempo,TimeSignature,Valence,song_cleaned
0,made for you,jake owen,32,20,2021,None,jake owen,0.7700,jake owen,0.581,...,0.000028,0.1110,-6.829,1,made for you,0.0268,82.125,4,0.337,made for you
1,made for you,jake owen,32,19,2021,None,jake owen,0.7700,jake owen,0.581,...,0.000028,0.1110,-6.829,1,made for you,0.0268,82.125,4,0.337,made for you
2,made for you,jake owen,32,18,2021,None,jake owen,0.7700,jake owen,0.581,...,0.000028,0.1110,-6.829,1,made for you,0.0268,82.125,4,0.337,made for you
3,made for you,jake owen,32,17,2021,None,jake owen,0.7700,jake owen,0.581,...,0.000028,0.1110,-6.829,1,made for you,0.0268,82.125,4,0.337,made for you
4,made for you,jake owen,32,16,2021,None,jake owen,0.7700,jake owen,0.581,...,0.000028,0.1110,-6.829,1,made for you,0.0268,82.125,4,0.337,made for you
5,made for you,jake owen,32,15,2021,None,jake owen,0.7700,jake owen,0.581,...,0.000028,0.1110,-6.829,1,made for you,0.0268,82.125,4,0.337,made for you
6,made for you,jake owen,32,14,2021,None,jake owen,0.7700,jake owen,0.581,...,0.000028,0.1110,-6.829,1,made for you,0.0268,82.125,4,0.337,made for you
7,interlude,j. cole,8,3,2021,None,j. cole,0.9360,j. cole,0.521,...,0.000168,0.1150,-14.619,0,interlude,0.1050,64.825,3,0.157,interlude
8,interlude,j. cole,8,2,2021,None,j. cole,0.9360,j. cole,0.521,...,0.000168,0.1150,-14.619,0,interlude,0.1050,64.825,3,0.157,interlude
9,made for you,jake owen,32,13,2021,None,jake owen,0.7700,jake owen,0.581,...,0.000028,0.1110,-6.829,1,made for you,0.0268,82.125,4,0.337,made for you


In [65]:
unique_songs = another_try_df.drop_duplicates(subset=['song_cleaned', 'artist_x'])
unique_songs['year'].max()

2021

In [20]:
updated_song_attributes_df['song_cleaned'] = updated_song_attributes_df['song'].str.replace(r'\s*\((feat\.?|featuring)[^)]*\)', '', regex=True).str.strip()
updated_song_attributes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154931 entries, 0 to 154930
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Acousticness      154931 non-null  float64
 1   artist            154931 non-null  object 
 2   Danceability      154931 non-null  float64
 3   Duration          154931 non-null  int64  
 4   Energy            154931 non-null  float64
 5   Explicit          154931 non-null  bool   
 6   Instrumentalness  154931 non-null  float64
 7   Liveness          154931 non-null  float64
 8   Loudness          154931 non-null  float64
 9   Mode              154931 non-null  int64  
 10  song              154931 non-null  object 
 11  Speechiness       154931 non-null  float64
 12  Tempo             154931 non-null  float64
 13  TimeSignature     154931 non-null  int64  
 14  Valence           154931 non-null  float64
 15  song_cleaned      154931 non-null  object 
dtypes: bool(1), float64(

In [52]:
artists_att_df = pd.merge(filtered_artist_df, updated_song_attributes_df, left_on=['song', 'main_artist'], right_on=['song_cleaned', 'artist'], how='inner')
artists_qualities_df

NameError: name 'artists_qualities_df' is not defined

In [56]:
range_charts_qual_df = pd.merge(range_updated_charts_df, updated_spotify_qualities_df, left_on=['song', 'main_artist'], right_on=['song', 'artist'])
range_charts_qual_df.head(50)

KeyError: 'artist'

In [59]:
chart_and_att_df = pd.merge(updated_charts_df, updated_song_attributes_df, left_on=['song', 'main_artist'], right_on=['song', 'artist'], how='inner')
#grouped_chart_and_att_df = chart_and_att_df.groupby('song').first().reset_index()
chart_and_att_df.head(50)

,song,artist_x,peak-rank,weeks-on-board,year,featured_artist,main_artist,Acousticness,artist_y,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Speechiness,Tempo,TimeSignature,Valence,song_cleaned
0,made for you,jake owen,32,20,2021,None,jake owen,0.7700,jake owen,0.581,238400,0.441,False,0.000028,0.1110,-6.829,1,0.0268,82.125,4,0.337,made for you
1,made for you,jake owen,32,19,2021,None,jake owen,0.7700,jake owen,0.581,238400,0.441,False,0.000028,0.1110,-6.829,1,0.0268,82.125,4,0.337,made for you
2,made for you,jake owen,32,18,2021,None,jake owen,0.7700,jake owen,0.581,238400,0.441,False,0.000028,0.1110,-6.829,1,0.0268,82.125,4,0.337,made for you
3,made for you,jake owen,32,17,2021,None,jake owen,0.7700,jake owen,0.581,238400,0.441,False,0.000028,0.1110,-6.829,1,0.0268,82.125,4,0.337,made for you
4,made for you,jake owen,32,16,2021,None,jake owen,0.7700,jake owen,0.581,238400,0.441,False,0.000028,0.1110,-6.829,1,0.0268,82.125,4,0.337,made for you
5,made for you,jake owen,32,15,2021,None,jake owen,0.7700,jake owen,0.581,238400,0.441,False,0.000028,0.1110,-6.829,1,0.0268,82.125,4,0.337,made for you
6,made for you,jake owen,32,14,2021,None,jake owen,0.7700,jake owen,0.581,238400,0.441,False,0.000028,0.1110,-6.829,1,0.0268,82.125,4,0.337,made for you
7,interlude,j. cole,8,3,2021,None,j. cole,0.9360,j. cole,0.521,99680,0.285,True,0.000168,0.1150,-14.619,0,0.1050,64.825,3,0.157,interlude
8,interlude,j. cole,8,2,2021,None,j. cole,0.9360,j. cole,0.521,99680,0.285,True,0.000168,0.1150,-14.619,0,0.1050,64.825,3,0.157,interlude
9,made for you,jake owen,32,13,2021,None,jake owen,0.7700,jake owen,0.581,238400,0.441,False,0.000028,0.1110,-6.829,1,0.0268,82.125,4,0.337,made for you


In [62]:
group_chart_and_att_df = chart_and_att_df.sort_values('peak-rank').drop_duplicates(subset=['song', 'main_artist'])
group_chart_and_att_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4963 entries, 18473 to 1164
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song              4963 non-null   object 
 1   artist_x          4963 non-null   object 
 2   peak-rank         4963 non-null   int64  
 3   weeks-on-board    4963 non-null   int64  
 4   year              4963 non-null   int32  
 5   featured_artist   783 non-null    object 
 6   main_artist       4963 non-null   object 
 7   Acousticness      4963 non-null   float64
 8   artist_y          4963 non-null   object 
 9   Danceability      4963 non-null   float64
 10  Duration          4963 non-null   int64  
 11  Energy            4963 non-null   float64
 12  Explicit          4963 non-null   bool   
 13  Instrumentalness  4963 non-null   float64
 14  Liveness          4963 non-null   float64
 15  Loudness          4963 non-null   float64
 16  Mode              4963 non-null   int64  
 

In [ ]:
chart_att_and_qual_df = pd.merge(grouped_chart_and_att_df, updated_spotify_qualities_df, on=['year', 'song'], how='inner')
chart_att_and_qual_df.info()

In [51]:
#New columns for Main and Featured Artists
featured_artists= ['featuring', '&', 'with']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

grouped_chart_and_att_df['featured_artist'] = grouped_chart_and_att_df['artist_x'].apply(extract_featured)
grouped_chart_and_att_df['main_artist'] = grouped_chart_and_att_df['artist_x'].apply(extract_main_artist)
grouped_chart_and_att_df['main_artist'].value_counts()

NameError: name 'grouped_chart_and_att_df' is not defined

In [ ]:
grouped_lyrics_df = lyrics_df.groupby('song').first().reset_index()
grouped_lyrics_df

In [ ]:
charts_df.info()

In [ ]:
charts_df['artist'].value_counts()

In [ ]:
lyrics_df = lyrics_df.rename(columns={'ranking': 'rank'})
spotify_qualities_df = spotify_qualities_df.rename(columns={'Time_Signature': 'time_signature', 'Track': 'song', 'Year': 'year', 'Danceability': 'danceability', 'Energy': 'energy', 'Key': 'key', 'Loudness': 'loudness', 'Mode': 'mode', 'Speechiness': 'speechiness', 'Liveness': 'liveness', 'Valence': 'valence', 'Tempo': 'tempo'}) 
spotify_qualities_df.info()

In [ ]:
charts_df['datetime'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['datetime'].dt.year

In [ ]:
charts_df.info()

In [ ]:
charts_and_lyrics_df = pd.merge(charts_df, lyrics_df, on=['song'], how='inner')
pd.set_option('display.max_columns', None)
charts_and_lyrics_df.info()

In [ ]:
range_charts_and_lyrics_df = charts_and_lyrics_df[
    (charts_and_lyrics_df['year'] >= 1999) &
    (charts_and_lyrics_df['year'] <= 2023)]
no_repeats_range_charts_and_lyrics_df = range_charts_and_lyrics_df.groupby('song').first().reset_index()
no_repeats_range_charts_and_lyrics_df.head(10)

In [ ]:
#charts_and_lyrics_qualities_df = pd.merge(charts_and_lyrics_df, spotify_qualities_df, on=['year', 'song', 'time_signature', 'danceability', 'energy', 'loudness', 'speechiness', 'liveness', 'valence', 'tempo'], how='inner')
#pd.set_option('display.max_columns', None)
#charts_and_lyrics_qualities_df= charts_and_lyrics_qualities_df['song'].nunique()
#charts_and_lyrics_qualities_df.head(30)

In [ ]:
artists = sorted(charts_and_lyrics_df['artist'].astype(str).unique().tolist())
#artists

In [ ]:
#New columns for Main and Featured Artists
featured_artists= ['Featuring', '&', 'With']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

charts_and_lyrics_df['featured_artist'] = charts_and_lyrics_df['artist'].apply(extract_featured)
charts_and_lyrics_df['main_artist'] = charts_and_lyrics_df['artist'].apply(extract_main_artist)
charts_and_lyrics_df['main_artist'].head(20)

In [ ]:
spotify_qualities_df.dtypes

In [ ]:
#lyrics_df['time_signature'].value_counts()
spotify_qualities_df['time_signature'].value_counts()

In [ ]:
charts_and_lyrics_df[charts_and_lyrics_df['time_signature'] == 5]

In [ ]:
spotify_qualities_df['year'].max()
charts_and_lyrics_df['year'].max()

In [ ]:
charts_and_lyrics_df.info()

In [ ]:
updated_charts_and_lyrics_df = no_repeats_range_charts_and_lyrics_df.combine_first(spotify_qualities_df.set_index('song')).reset_index()
updated_charts_and_lyrics_df.head()